<a href="https://colab.research.google.com/github/seravee08/GPU-ECC-GPU-Computation-of-the-Euler-Characteristic-Curve-for-Imaging-Data/blob/main/GPU_ECC_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install GPU-ECC
!pip install https://github.com/seravee08/GPU-ECC-GPU-Computation-of-the-Euler-Characteristic-Curve-for-Imaging-Data/raw/main/releases/gpuecc-0.1.0-cp312-cp312-linux_x86_64.whl

# Download data
!wget "https://raw.githubusercontent.com/seravee08/GPU-ECC-GPU-Computation-of-the-Euler-Characteristic-Curve-for-Imaging-Data/main/data/2D_128_0.dat" \
     -O /content/sample_data/2D_128_0.dat
!wget "https://raw.githubusercontent.com/seravee08/GPU-ECC-GPU-Computation-of-the-Euler-Characteristic-Curve-for-Imaging-Data/main/data/3D_128_0.dat" \
     -O /content/sample_data/3D_128_0.dat


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.7 MB/s eta 0:00:00
--2026-02-11 04:56:05--  https://raw.githubusercontent.com/seravee08/GPU-ECC-GPU-Computation-of-the-Euler-Characteristic-Curve-for-Imaging-Data/main/data/2D_128_0.dat
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 65536 (64K) [application/octet-stream]
Saving to: ‘/content/sample_data/2D_128_0.dat’

/content/sample_dat 100%[===================>]  64.00K  --.-KB/s    in 0.001s  

2026-02-11 04:56:06 (57.9 MB/s) - ‘/content/sample_data/2D_128_0.dat’ saved [65536/65536]

--2026-02-11 04:56:06--  https://raw.githubusercontent.com/seravee08/GPU-ECC-GPU-Computation-of-the-Euler-Characteristic-Curve-for-Imaging-Data/main/data/3D_128_0.dat
Resolving raw.githubusercontent.com (raw.githubu

In [2]:
# If in Colab, hardware accelerator has to be GPU

import time
import numpy as np
import gpuecc

def read3d(filename: str, h: int, w: int, d: int, input_dtype=np.float32) -> np.ndarray:
    """
    Reads d*h*w elements of type input_dtype from a binary file and returns float32 array.
    Returns:
        arr: shape (d*h*w,), dtype float32
    """
    total = d * h * w
    data = np.fromfile(filename, dtype=input_dtype, count=total)
    if data.size != total:
        raise RuntimeError(f"File read failed or file too short: expected {total} elements, got {data.size}")
    return data.astype(np.float32, copy=False)

def read2d(filename: str, h: int, w: int, input_dtype=np.float32) -> np.ndarray:
    """
    Reads h*w elements of type input_dtype from a binary file and returns float32 array.
    Returns:
        arr: shape (h*w,), dtype float32
    """
    total = h * w
    data = np.fromfile(filename, dtype=input_dtype, count=total)
    if data.size != total:
        raise RuntimeError(f"File read failed or file too short: expected {total} elements, got {data.size}")
    return data.astype(np.float32, copy=False)

def save_results(arr, out_filename):
    a = np.asarray(arr)
    if a.ndim != 2 or a.shape[1] != 2:
        raise ValueError(f"Expected Nx2 array, got {a.shape}")
    with open(out_filename, "w", encoding="utf-8") as f:
        for x, y in a:
            f.write(f"{float(x):g} {int(round(float(y)))}\n")

# ---------- config ----------
height, width, depth = 128, 128, 128
path_2d = "/content/sample_data/2D_128_0.dat"
path_3d = "/content/sample_data/3D_128_0.dat"

# Create a 2D gpu-ecc
t0 = time.perf_counter()
ecc2d = gpuecc.GPUECC(height, width, 0)
arr2d = ecc2d.run_frmFile(path_2d) # run from file
t1 = time.perf_counter()
print(f"GPU-ECC time: {(t1 - t0):.4f} s")
print("2D results has shape:", arr2d.shape)

# Create a 3D gpu-ecc
t0 = time.perf_counter()
ecc3d  = gpuecc.GPUECC(height, width, depth)
data3d = read3d(path_3d, height, width, depth, input_dtype=np.float32)
arr3d  = ecc3d.run_frmArr(data3d) # run from numpy array
t1 = time.perf_counter()
print(f"GPU-ECC time: {(t1 - t0):.4f} s")
print("3D results has shape:", arr3d.shape)

GPU-ECC time: 0.3583 s
2D results has shape: (412, 2)
GPU-ECC time: 0.1510 s
3D results has shape: (622, 2)
